In [21]:
import random

# Number of Players and Rounds
num_players = 10
num_rounds = 5
num_simulations = 1000

# Sample Players with Elo ratings (example data)
players = [{'name': f'Player {i}', 'elo': random.randint(1400, 2600), 'points': 0} for i in range(num_players)]



def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2 """
    return 1 / (1 + 10 ** ((player2['elo'] - player1['elo']) / 400))

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()
    if result < win_prob:
        player1['points'] += 1  # Player 1 wins
    elif result < win_prob + (1 - win_prob) / 2:
        player1['points'] += 0.5
        player2['points'] += 0.5  # Draw
    else:
        player2['points'] += 1  # Player 2 wins

def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            # Avoid pairing the same players again (simple check)
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings


In [22]:
def simulate_tournament():
    """ Simulate a single Swiss-system tournament """
    # Reset points and opponents for each player
    for player in players:
        player['points'] = 0
        player['opponents'] = []

    # Simulate rounds
    for _ in range(num_rounds):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2)

    # Determine winner (basic - most points)
    winner = max(players, key=lambda x: x['points'])
    return winner['name']

# Run multiple simulations and tally wins
tournament_wins = {player['name']: 0 for player in players}
for _ in range(num_simulations):
    winner = simulate_tournament()
    tournament_wins[winner] += 1

# Print probabilities of winning the tournament
for player in players:
    probability = tournament_wins[player['name']] / num_simulations
    print(f"{player['name']} winning probability: {probability:.2f}")


Player 0 winning probability: 0.52
Player 1 winning probability: 0.00
Player 2 winning probability: 0.09
Player 3 winning probability: 0.00
Player 4 winning probability: 0.04
Player 5 winning probability: 0.00
Player 6 winning probability: 0.02
Player 7 winning probability: 0.00
Player 8 winning probability: 0.03
Player 9 winning probability: 0.30


In [1]:
import random

# Number of Players, Rounds, and Simulations
num_players = 10
num_rounds = 5
num_simulations = 1000
k_factor = 20  # K-factor for Elo rating updates

# Initialize Players with Elo ratings
players = [{'name': f'Player {i}', 'elo': random.randint(1400, 2600), 'points': 0} for i in range(num_players)]

def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2 """
    return 1 / (1 + 10 ** ((player2['elo'] - player1['elo']) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings after a game """
    expected_win_winner = calculate_win_probability(winner, loser)
    expected_win_loser = calculate_win_probability(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += k_factor * (score_winner - expected_win_winner)
    loser['elo'] += k_factor * (score_loser - expected_win_loser)

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()
    if result < win_prob:
        player1['points'] += 1  # Player 1 wins
        update_elo_ratings(player1, player2)
    elif result < win_prob + (1 - win_prob) / 2:
        player1['points'] += 0.5
        player2['points'] += 0.5  # Draw
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['points'] += 1  # Player 2 wins
        update_elo_ratings(player2, player1)

def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

def print_round_statistics(players, round_number):
    """ Print statistics after each round """
    print(f"\nAfter Round {round_number}:")
    for player in players:
        print(f"{player['name']} - Points: {player['points']}, Elo: {player['elo']:.2f}")

def simulate_tournament():
    """ Simulate a single Swiss-system tournament """
    # Reset points and opponents for each player
    for player in players:
        player['points'] = 0
        player['opponents'] = []
        player['elo'] = random.randint(1400, 2600)  # Reset Elo for each simulation

    # Simulate rounds
    for round_number in range(1, num_rounds + 1):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2)
        print_round_statistics(players, round_number)

    # Determine winner (most points)
    winner = max(players, key=lambda x: x['points'])
    return winner['name']

# Simulate and Print Results
for _ in range(num_simulations):
    simulate_tournament()



After Round 1:
Player 0 - Points: 1, Elo: 2520.01
Player 1 - Points: 0, Elo: 2134.99
Player 2 - Points: 0, Elo: 1936.46
Player 3 - Points: 1, Elo: 2560.54
Player 4 - Points: 0, Elo: 1726.17
Player 5 - Points: 1, Elo: 2273.83
Player 6 - Points: 1, Elo: 1647.09
Player 7 - Points: 0, Elo: 1662.91
Player 8 - Points: 1, Elo: 2365.82
Player 9 - Points: 0, Elo: 2478.18

After Round 2:
Player 0 - Points: 1, Elo: 2511.17
Player 1 - Points: 0, Elo: 2130.81
Player 2 - Points: 1, Elo: 1941.05
Player 3 - Points: 2, Elo: 2569.38
Player 4 - Points: 0, Elo: 1721.58
Player 5 - Points: 2, Elo: 2274.36
Player 6 - Points: 1, Elo: 1646.56
Player 7 - Points: 0.5, Elo: 1672.73
Player 8 - Points: 2, Elo: 2370.01
Player 9 - Points: 0.5, Elo: 2468.36

After Round 3:
Player 0 - Points: 2, Elo: 2517.32
Player 1 - Points: 0.5, Elo: 2122.14
Player 2 - Points: 2, Elo: 1944.15
Player 3 - Points: 2.5, Elo: 2562.47
Player 4 - Points: 0, Elo: 1721.31
Player 5 - Points: 2.5, Elo: 2281.26
Player 6 - Points: 1, Elo: 1643.

Player 0 - Points: 2, Elo: 1899.18
Player 1 - Points: 1.5, Elo: 1820.33
Player 2 - Points: 2, Elo: 2579.68
Player 3 - Points: 1, Elo: 2112.84
Player 4 - Points: 3, Elo: 2571.54
Player 5 - Points: 1.5, Elo: 2103.91
Player 6 - Points: 0, Elo: 1633.55
Player 7 - Points: 2, Elo: 1816.77
Player 8 - Points: 0.5, Elo: 1465.07
Player 9 - Points: 1.5, Elo: 1772.12

After Round 4:
Player 0 - Points: 2, Elo: 1898.77
Player 1 - Points: 2.5, Elo: 1828.95
Player 2 - Points: 3, Elo: 2579.93
Player 3 - Points: 1, Elo: 2102.59
Player 4 - Points: 4, Elo: 2571.95
Player 5 - Points: 2.5, Elo: 2114.17
Player 6 - Points: 0.5, Elo: 1629.05
Player 7 - Points: 2, Elo: 1816.52
Player 8 - Points: 1.0, Elo: 1469.57
Player 9 - Points: 1.5, Elo: 1763.50

After Round 5:
Player 0 - Points: 3, Elo: 1906.45
Player 1 - Points: 2.5, Elo: 1828.68
Player 2 - Points: 4, Elo: 2581.21
Player 3 - Points: 2, Elo: 2103.10
Player 4 - Points: 5, Elo: 2572.22
Player 5 - Points: 2.5, Elo: 2112.89
Player 6 - Points: 0.5, Elo: 1629.05

Player 1 - Points: 0, Elo: 1507.90
Player 2 - Points: 1, Elo: 2268.72
Player 3 - Points: 0, Elo: 1854.28
Player 4 - Points: 1, Elo: 2499.19
Player 5 - Points: 0, Elo: 1687.81
Player 6 - Points: 0.5, Elo: 1687.71
Player 7 - Points: 0.5, Elo: 2399.29
Player 8 - Points: 1, Elo: 1812.96
Player 9 - Points: 0, Elo: 1423.04

After Round 2:
Player 0 - Points: 2, Elo: 2439.67
Player 1 - Points: 1, Elo: 1522.66
Player 2 - Points: 1, Elo: 2263.15
Player 3 - Points: 0, Elo: 1853.45
Player 4 - Points: 2, Elo: 2499.56
Player 5 - Points: 1, Elo: 1691.39
Player 6 - Points: 0.5, Elo: 1672.95
Player 7 - Points: 1.5, Elo: 2400.13
Player 8 - Points: 1, Elo: 1812.58
Player 9 - Points: 0, Elo: 1419.47

After Round 3:
Player 0 - Points: 3, Elo: 2451.37
Player 1 - Points: 1, Elo: 1522.53
Player 2 - Points: 2, Elo: 2263.86
Player 3 - Points: 1, Elo: 1854.97
Player 4 - Points: 2, Elo: 2487.86
Player 5 - Points: 1, Elo: 1690.67
Player 6 - Points: 1.5, Elo: 1686.76
Player 7 - Points: 2.5, Elo: 2400.25
Player 8 - 

Player 7 - Points: 1.0, Elo: 1845.06
Player 8 - Points: 3, Elo: 2115.10
Player 9 - Points: 2.5, Elo: 1861.51

After Round 5:
Player 0 - Points: 2.5, Elo: 1956.20
Player 1 - Points: 2, Elo: 1901.15
Player 2 - Points: 4.5, Elo: 2330.68
Player 3 - Points: 0.5, Elo: 1486.13
Player 4 - Points: 4, Elo: 2437.89
Player 5 - Points: 1.0, Elo: 1622.44
Player 6 - Points: 2.5, Elo: 1992.65
Player 7 - Points: 2.0, Elo: 1847.33
Player 8 - Points: 3.5, Elo: 2120.73
Player 9 - Points: 2.5, Elo: 1860.81

After Round 1:
Player 0 - Points: 0.5, Elo: 1430.99
Player 1 - Points: 0.5, Elo: 1922.01
Player 2 - Points: 0, Elo: 2419.20
Player 3 - Points: 1, Elo: 1662.80
Player 4 - Points: 1, Elo: 2179.00
Player 5 - Points: 0, Elo: 2020.00
Player 6 - Points: 0, Elo: 2161.45
Player 7 - Points: 1, Elo: 2594.55
Player 8 - Points: 1, Elo: 2363.34
Player 9 - Points: 0, Elo: 1658.66

After Round 2:
Player 0 - Points: 0.5, Elo: 1430.92
Player 1 - Points: 1.5, Elo: 1934.76
Player 2 - Points: 1, Elo: 2419.27
Player 3 - Poi

Player 0 - Points: 2, Elo: 2265.79
Player 1 - Points: 1, Elo: 1591.07
Player 2 - Points: 3.5, Elo: 2496.82
Player 3 - Points: 2, Elo: 2302.20
Player 4 - Points: 1.5, Elo: 2092.57
Player 5 - Points: 2.5, Elo: 2315.04
Player 6 - Points: 1, Elo: 2073.56
Player 7 - Points: 3, Elo: 2547.90
Player 8 - Points: 0, Elo: 1982.73
Player 9 - Points: 3.5, Elo: 2480.31

After Round 5:
Player 0 - Points: 2.5, Elo: 2266.84
Player 1 - Points: 1, Elo: 1591.07
Player 2 - Points: 3.5, Elo: 2488.28
Player 3 - Points: 2.5, Elo: 2301.16
Player 4 - Points: 2.5, Elo: 2102.02
Player 5 - Points: 2.5, Elo: 2309.47
Player 6 - Points: 1, Elo: 2064.11
Player 7 - Points: 4, Elo: 2556.44
Player 8 - Points: 0, Elo: 1982.73
Player 9 - Points: 4.5, Elo: 2485.88

After Round 1:
Player 0 - Points: 1, Elo: 1856.38
Player 1 - Points: 0, Elo: 1401.62
Player 2 - Points: 1, Elo: 1908.46
Player 3 - Points: 0, Elo: 1727.54
Player 4 - Points: 0, Elo: 2110.29
Player 5 - Points: 1, Elo: 1683.71
Player 6 - Points: 1, Elo: 2377.99
Pla

...

Player 9 - Points: 2.0, Elo: 1971.01

After Round 5:
Player 0 - Points: 1, Elo: 1933.59
Player 1 - Points: 2.5, Elo: 2243.11
Player 2 - Points: 3, Elo: 2109.36
Player 3 - Points: 2, Elo: 1617.31
Player 4 - Points: 1, Elo: 1637.31
Player 5 - Points: 4, Elo: 2065.72
Player 6 - Points: 0.5, Elo: 1610.89
Player 7 - Points: 4.5, Elo: 2489.71
Player 8 - Points: 2.5, Elo: 1554.80
Player 9 - Points: 3.0, Elo: 1980.20

After Round 1:
Player 0 - Points: 0, Elo: 1415.95
Player 1 - Points: 1, Elo: 2463.05
Player 2 - Points: 0.5, Elo: 2017.08
Player 3 - Points: 0.5, Elo: 2390.92
Player 4 - Points: 0.5, Elo: 1624.75
Player 5 - Points: 0.5, Elo: 2366.25
Player 6 - Points: 1, Elo: 2492.14
Player 7 - Points: 0, Elo: 2476.86
Player 8 - Points: 1, Elo: 1773.40
Player 9 - Points: 0, Elo: 1733.60

After Round 2:
Player 0 - Points: 0, Elo: 1415.87
Player 1 - Points: 1.5, Elo: 2463.88
Player 2 - Points: 1.0, Elo: 2011.03
Player 3 - Points: 1.5, Elo: 2391.16
Player 4 - Points: 0.5, Elo: 1624.51
Player 5 - Poi

Player 4 - Points: 0, Elo: 1919.92
Player 5 - Points: 1.5, Elo: 2381.07
Player 6 - Points: 1.5, Elo: 2461.51
Player 7 - Points: 2.0, Elo: 2164.07
Player 8 - Points: 0.5, Elo: 1758.58
Player 9 - Points: 2.0, Elo: 2514.81

After Round 4:
Player 0 - Points: 1, Elo: 1869.48
Player 1 - Points: 4, Elo: 2325.89
Player 2 - Points: 1.5, Elo: 2350.58
Player 3 - Points: 3, Elo: 2485.98
Player 4 - Points: 0, Elo: 1905.59
Player 5 - Points: 2.5, Elo: 2390.46
Player 6 - Points: 2.5, Elo: 2462.15
Player 7 - Points: 2.0, Elo: 2158.29
Player 8 - Points: 1.5, Elo: 1772.92
Player 9 - Points: 2.0, Elo: 2503.67

After Round 5:
Player 0 - Points: 1, Elo: 1856.77
Player 1 - Points: 4, Elo: 2319.62
Player 2 - Points: 2.0, Elo: 2345.55
Player 3 - Points: 4, Elo: 2493.30
Player 4 - Points: 0, Elo: 1904.97
Player 5 - Points: 2.5, Elo: 2383.15
Player 6 - Points: 3.5, Elo: 2468.42
Player 7 - Points: 2.5, Elo: 2163.32
Player 8 - Points: 2.5, Elo: 1785.63
Player 9 - Points: 3.0, Elo: 2504.29

After Round 1:
Player 0

Player 0 - Points: 1.5, Elo: 1891.59
Player 1 - Points: 0, Elo: 1718.25
Player 2 - Points: 0.5, Elo: 1929.43
Player 3 - Points: 1.5, Elo: 2163.28
Player 4 - Points: 0.5, Elo: 2183.74
Player 5 - Points: 1.5, Elo: 2224.58
Player 6 - Points: 1, Elo: 1788.15
Player 7 - Points: 1.5, Elo: 2570.43
Player 8 - Points: 2, Elo: 2247.30
Player 9 - Points: 0, Elo: 1557.26

After Round 3:
Player 0 - Points: 1.5, Elo: 1889.31
Player 1 - Points: 0.5, Elo: 1713.92
Player 2 - Points: 0.5, Elo: 1928.94
Player 3 - Points: 2.0, Elo: 2165.02
Player 4 - Points: 0.5, Elo: 2165.60
Player 5 - Points: 2.0, Elo: 2222.83
Player 6 - Points: 2, Elo: 1806.29
Player 7 - Points: 2.5, Elo: 2570.91
Player 8 - Points: 3, Elo: 2249.59
Player 9 - Points: 0.5, Elo: 1561.59

After Round 4:
Player 0 - Points: 1.5, Elo: 1886.75
Player 1 - Points: 1.5, Elo: 1729.42
Player 2 - Points: 0.5, Elo: 1913.44
Player 3 - Points: 3.0, Elo: 2167.28
Player 4 - Points: 1.5, Elo: 2166.20
Player 5 - Points: 3.0, Elo: 2225.39
Player 6 - Points:

Player 1 - Points: 3.0, Elo: 2209.61
Player 2 - Points: 1, Elo: 2004.40
Player 3 - Points: 3.5, Elo: 2344.31
Player 4 - Points: 4.5, Elo: 2330.84
Player 5 - Points: 3.0, Elo: 2281.74
Player 6 - Points: 3, Elo: 2102.29
Player 7 - Points: 2.5, Elo: 2034.50
Player 8 - Points: 0.5, Elo: 1704.24
Player 9 - Points: 1.5, Elo: 1927.43

After Round 1:
Player 0 - Points: 0, Elo: 1737.00
Player 1 - Points: 1, Elo: 2250.00
Player 2 - Points: 0.5, Elo: 1689.02
Player 3 - Points: 0.5, Elo: 1828.98
Player 4 - Points: 1, Elo: 2306.12
Player 5 - Points: 0, Elo: 1420.88
Player 6 - Points: 1, Elo: 2550.04
Player 7 - Points: 0, Elo: 1470.96
Player 8 - Points: 0.5, Elo: 1469.16
Player 9 - Points: 0.5, Elo: 1614.84

After Round 2:
Player 0 - Points: 0.5, Elo: 1733.62
Player 1 - Points: 2, Elo: 2261.60
Player 2 - Points: 0.5, Elo: 1688.88
Player 3 - Points: 1.5, Elo: 1831.22
Player 4 - Points: 1, Elo: 2294.52
Player 5 - Points: 0.5, Elo: 1422.31
Player 6 - Points: 2, Elo: 2550.18
Player 7 - Points: 0.5, Elo:

Player 8 - Points: 0.5, Elo: 2541.49
Player 9 - Points: 0.5, Elo: 2590.51

After Round 2:
Player 0 - Points: 1, Elo: 1810.66
Player 1 - Points: 0, Elo: 1561.93
Player 2 - Points: 2, Elo: 2519.78
Player 3 - Points: 0, Elo: 1861.26
Player 4 - Points: 1, Elo: 2103.29
Player 5 - Points: 1, Elo: 1900.91
Player 6 - Points: 2, Elo: 2456.36
Player 7 - Points: 0, Elo: 2016.44
Player 8 - Points: 1.5, Elo: 2541.56
Player 9 - Points: 1.5, Elo: 2590.81

After Round 3:
Player 0 - Points: 1, Elo: 1810.37
Player 1 - Points: 0, Elo: 1559.44
Player 2 - Points: 2, Elo: 2507.97
Player 3 - Points: 1, Elo: 1875.45
Player 4 - Points: 1, Elo: 2102.15
Player 5 - Points: 2, Elo: 1903.40
Player 6 - Points: 3, Elo: 2468.17
Player 7 - Points: 0, Elo: 2002.25
Player 8 - Points: 2.5, Elo: 2541.85
Player 9 - Points: 2.5, Elo: 2591.95

After Round 4:
Player 0 - Points: 1, Elo: 1802.99
Player 1 - Points: 0, Elo: 1557.99
Player 2 - Points: 2.5, Elo: 2510.34
Player 3 - Points: 2, Elo: 1891.18
Player 4 - Points: 1, Elo: 2

In [29]:
import random

# Number of Players, Rounds, and Simulations
num_players = 10
num_rounds = 5
num_simulations = 400
k_factor = 20  # K-factor for Elo rating updates

# Initialize Players with Elo ratings
players = [{'name': f'Player {i}', 'elo': random.randint(1400, 2600), 'points': 0} for i in range(num_players)]

def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2 """
    return 1 / (1 + 10 ** ((player2['elo'] - player1['elo']) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings after a game """
    expected_win_winner = calculate_win_probability(winner, loser)
    expected_win_loser = calculate_win_probability(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += k_factor * (score_winner - expected_win_winner)
    loser['elo'] += k_factor * (score_loser - expected_win_loser)

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()
    if result < win_prob:
        player1['points'] += 1  # Player 1 wins
        update_elo_ratings(player1, player2)
    elif result < win_prob + (1 - win_prob) / 2:
        player1['points'] += 0.5
        player2['points'] += 0.5  # Draw
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['points'] += 1  # Player 2 wins
        update_elo_ratings(player2, player1)

def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

def print_final_round_statistics(players, simulation_number):
    """ Print statistics for the final round """
    if 390 <= simulation_number <= 400:
        print(f"\nSimulation {simulation_number} - Final Round Results:")
        for player in players:
            # Print probabilities of winning the tournament

            probability = tournament_wins[player['name']] / num_simulations
#           print(f"{player['name']} winning probability: {probability:.2f}")
            print(f"{player['name']} - Points: {player['points']}, Elo: {player['elo']:.2f} winning probability: {probability:.2f}")

def simulate_tournament(simulation_number):
    """ Simulate a single Swiss-system tournament """
    # Reset points and opponents for each player
    for player in players:
        player['points'] = 0
        player['opponents'] = []
        player['elo'] = random.randint(1400, 2600)  # Reset Elo for each simulation

    # Simulate rounds
    for round_number in range(1, num_rounds + 1):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2)
        # update elo ratings after actual outcome         

    # Print final round statistics for specified simulations
    if round_number == num_rounds:
        print_final_round_statistics(players, simulation_number)

# Simulate and Print Results
for simulation_number in range(1, num_simulations + 1):
    simulate_tournament(simulation_number)



Simulation 390 - Final Round Results:
Player 0 - Points: 3, Elo: 2014.10 winning probability: 1.29
Player 1 - Points: 1, Elo: 1570.92 winning probability: 0.01
Player 2 - Points: 3.5, Elo: 2099.65 winning probability: 0.23
Player 3 - Points: 1, Elo: 1993.76 winning probability: 0.00
Player 4 - Points: 2.5, Elo: 1956.72 winning probability: 0.09
Player 5 - Points: 0, Elo: 1913.84 winning probability: 0.01
Player 6 - Points: 2.0, Elo: 1972.64 winning probability: 0.04
Player 7 - Points: 4.5, Elo: 2535.00 winning probability: 0.00
Player 8 - Points: 2.5, Elo: 2197.72 winning probability: 0.08
Player 9 - Points: 4, Elo: 2195.66 winning probability: 0.76

Simulation 391 - Final Round Results:
Player 0 - Points: 2.5, Elo: 1599.74 winning probability: 1.29
Player 1 - Points: 1.5, Elo: 1498.82 winning probability: 0.01
Player 2 - Points: 3.5, Elo: 2550.03 winning probability: 0.23
Player 3 - Points: 0.5, Elo: 1432.34 winning probability: 0.00
Player 4 - Points: 2.5, Elo: 1851.79 winning proba

In [30]:
import random

# Number of Players, Rounds, and Simulations
num_players = 10
num_rounds = 5
num_simulations = 1000
k_factor = 20  # K-factor for Elo rating updates

# Initialize Players with Elo ratings
players = [{'name': f'Player {i}', 'elo': random.randint(1400, 2600), 'points': 0, 'wins': 0} for i in range(num_players)]

def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2 """
    return 1 / (1 + 10 ** ((player2['elo'] - player1['elo']) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings after a game """
    expected_win_winner = calculate_win_probability(winner, loser)
    expected_win_loser = calculate_win_probability(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += k_factor * (score_winner - expected_win_winner)
    loser['elo'] += k_factor * (score_loser - expected_win_loser)

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()
    if result < win_prob:
        player1['points'] += 1  # Player 1 wins
        update_elo_ratings(player1, player2)
    elif result < win_prob + (1 - win_prob) / 2:
        player1['points'] += 0.5
        player2['points'] += 0.5  # Draw
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['points'] += 1  # Player 2 wins
        update_elo_ratings(player2, player1)

def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

def print_final_round_statistics(players, simulation_number):
    """ Print statistics for the final round """
    if 990 <= simulation_number <= 1000:
        print(f"\nSimulation {simulation_number} - Final Round Results:")
        for player in players:
            winning_probability = player['wins'] / num_simulations
            print(f"{player['name']} - Points: {player['points']}, Elo: {player['elo']:.2f}, Winning Probability: {winning_probability:.2f}")

def simulate_tournament(simulation_number):
    """ Simulate a single Swiss-system tournament """
    # Reset points and opponents for each player
    for player in players:
        player['points'] = 0
        player['opponents'] = []

    # Simulate rounds
    for round_number in range(1, num_rounds + 1):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2)

    # Determine winner and update wins
    winner = max(players, key=lambda x: x['points'])
    winner['wins'] += 1

    # Print final round statistics for specified simulations
    if round_number == num_rounds:
        print_final_round_statistics(players, simulation_number)

# Simulate and Print Results
for simulation_number in range(1, num_simulations + 1):
    simulate_tournament(simulation_number)

# Reset wins count for next simulation
for player in players:
    player['wins'] = 0



Simulation 990 - Final Round Results:
Player 0 - Points: 5, Elo: 2901.79, Winning Probability: 0.79
Player 1 - Points: 1, Elo: 1457.79, Winning Probability: 0.00
Player 2 - Points: 2.5, Elo: 1970.77, Winning Probability: 0.06
Player 3 - Points: 3.0, Elo: 2028.35, Winning Probability: 0.00
Player 4 - Points: 4, Elo: 2078.52, Winning Probability: 0.11
Player 5 - Points: 2, Elo: 1836.34, Winning Probability: 0.00
Player 6 - Points: 3.0, Elo: 1986.12, Winning Probability: 0.02
Player 7 - Points: 2, Elo: 1851.67, Winning Probability: 0.00
Player 8 - Points: 1.5, Elo: 2019.37, Winning Probability: 0.01
Player 9 - Points: 0, Elo: 1053.26, Winning Probability: 0.00

Simulation 991 - Final Round Results:
Player 0 - Points: 5, Elo: 2902.30, Winning Probability: 0.79
Player 1 - Points: 1, Elo: 1457.89, Winning Probability: 0.00
Player 2 - Points: 3, Elo: 1992.08, Winning Probability: 0.06
Player 3 - Points: 3.0, Elo: 2028.91, Winning Probability: 0.00
Player 4 - Points: 3, Elo: 2084.80, Winning 

In [36]:
import random

# Number of Players, Rounds, and Simulations
num_players = 10
num_rounds = 5
num_simulations = 10000
k_factor = 30  # Higher K-factor for more responsive Elo rating updates

# Initialize Players with Elo ratings
players = [{'name': f'Player {i}', 'elo': random.randint(1400, 2000), 'points': 0, 'wins': 0, 'current_form': 0} for i in range(num_players)]

def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2, considering current form """
    # Adjusting Elo ratings based on current form
    adjusted_elo1 = player1['elo'] + player1['current_form']
    adjusted_elo2 = player2['elo'] + player2['current_form']
    return 1 / (1 + 10 ** ((adjusted_elo2 - adjusted_elo1) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings and current form after a game """
    expected_win_winner = calculate_win_probability(winner, loser)
    expected_win_loser = calculate_win_probability(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += k_factor * (score_winner - expected_win_winner)
    loser['elo'] += k_factor * (score_loser - expected_win_loser)

    # Update current form based on recent game outcome
    winner['current_form'] += k_factor * 0.5 if not draw else -k_factor * 0.2
    loser['current_form'] += -k_factor * 0.5 if not draw else k_factor * 0.2

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()
    if result < win_prob:
        player1['points'] += 1  # Player 1 wins
        update_elo_ratings(player1, player2)
    elif result < win_prob + (1 - win_prob) / 2:
        player1['points'] += 0.5
        player2['points'] += 0.5  # Draw
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['points'] += 1  # Player 2 wins
        update_elo_ratings(player2, player1)

def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

def print_final_round_statistics(players, simulation_number):
    """ Print statistics for the final round """
    if 9990 <= simulation_number <= 10000:
        print(f"\nSimulation {simulation_number} - Final Round Results:")
        for player in players:
            winning_probability = player['wins'] / num_simulations
            print(f"{player['name']} - Points: {player['points']}, Elo: {player['elo']:.2f}, Winning Probability: {winning_probability:.2f}")

def simulate_tournament(simulation_number):
    """ Simulate a single Swiss-system tournament """
    # Reset points, opponents, and current form for each player
    for player in players:
        player['points'] = 0
        player['opponents'] = []
        player['current_form'] = 0
        player['elo'] = random.randint(1400, 2000)  # Reset Elo for each simulation

    # Simulate rounds
    for round_number in range(1, num_rounds + 1):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2)

    # Determine winner and update wins
    winner = max(players, key=lambda x: x['points'])
    winner['wins'] += 1

    # Print final round statistics for specified simulations
    if round_number == num_rounds:
        print_final_round_statistics(players, simulation_number)

# Simulate and Print Results
for simulation_number in range(1, num_simulations + 1):
    simulate_tournament(simulation_number)



Simulation 9990 - Final Round Results:
Player 0 - Points: 2, Elo: 1865.93, Winning Probability: 0.19
Player 1 - Points: 3, Elo: 1960.75, Winning Probability: 0.13
Player 2 - Points: 0.5, Elo: 1529.58, Winning Probability: 0.12
Player 3 - Points: 4.5, Elo: 1936.63, Winning Probability: 0.08
Player 4 - Points: 1.0, Elo: 1666.22, Winning Probability: 0.11
Player 5 - Points: 2, Elo: 1954.33, Winning Probability: 0.07
Player 6 - Points: 3, Elo: 1918.40, Winning Probability: 0.09
Player 7 - Points: 2.5, Elo: 1797.20, Winning Probability: 0.06
Player 8 - Points: 1.5, Elo: 1636.23, Winning Probability: 0.08
Player 9 - Points: 4.0, Elo: 1809.74, Winning Probability: 0.05

Simulation 9991 - Final Round Results:
Player 0 - Points: 3, Elo: 1874.72, Winning Probability: 0.19
Player 1 - Points: 2.5, Elo: 1948.65, Winning Probability: 0.13
Player 2 - Points: 0.5, Elo: 1395.86, Winning Probability: 0.12
Player 3 - Points: 3.5, Elo: 1477.66, Winning Probability: 0.08
Player 4 - Points: 5, Elo: 1915.94

In [38]:
import random

# Number of Players, Rounds, and Simulations
num_players = 10
num_rounds = 5
num_simulations = 1000
k_factor = 30  # Higher K-factor for more responsive Elo rating updates

# Initialize Players with Elo ratings
players = [{'name': f'Player {i}', 'initial_elo': random.randint(1400, 2600), 'elo': 0, 'points': 0, 'total_points': 0, 'wins': 0, 'current_form': 0} for i in range(num_players)]

def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2, considering current form """
    adjusted_elo1 = player1['elo'] + player1['current_form']
    adjusted_elo2 = player2['elo'] + player2['current_form']
    return 1 / (1 + 10 ** ((adjusted_elo2 - adjusted_elo1) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings and current form after a game """
    expected_win_winner = calculate_win_probability(winner, loser)
    expected_win_loser = calculate_win_probability(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += k_factor * (score_winner - expected_win_winner)
    loser['elo'] += k_factor * (score_loser - expected_win_loser)

    # Update current form based on recent game outcome
    # result of all parameter(win, loser, draw=True/NA) should be actual result
    winner['current_form'] += k_factor * 0.5 if not draw else -k_factor * 0.2
    loser['current_form'] += -k_factor * 0.5 if not draw else k_factor * 0.2

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()
    if result < win_prob:
        player1['points'] += 1  # Player 1 wins
        update_elo_ratings(player1, player2)
    elif result < win_prob + (1 - win_prob) / 2:
        player1['points'] += 0.5
        player2['points'] += 0.5  # Draw
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['points'] += 1  # Player 2 wins
        update_elo_ratings(player2, player1)

def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

def simulate_tournament():
    """ Simulate a single Swiss-system tournament """
    # Reset points and opponents for each player
    for player in players:
        player['points'] = 0
        player['opponents'] = []
        player['current_form'] = 0
        player['elo'] = player['initial_elo']  # Reset Elo to initial Elo for each simulation

    # Simulate rounds
    for round_number in range(1, num_rounds + 1):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2)

    # Determine winner and update wins
    winner = max(players, key=lambda x: x['points'])
    winner['wins'] += 1

    # Accumulate total points for each player
    for player in players:
        player['total_points'] += player['points']

# Simulate all tournaments
for _ in range(1, num_simulations + 1):
    simulate_tournament()

# Calculate and print final results after all simulations
print("\nFinal Tournament Results after all simulations:")
for player in players:
    winning_probability = player['wins'] / num_simulations
    average_points = player['total_points'] / num_simulations
    print(f"{player['name']} - Initial Elo: {player['initial_elo']}, Final Elo: {player['elo']:.2f}, Average Points: {average_points:.2f}, Winning Probability: {winning_probability:.2f}")



Final Tournament Results after all simulations:
Player 0 - Initial Elo: 1837, Final Elo: 1842.54, Average Points: 2.12, Winning Probability: 0.01
Player 1 - Initial Elo: 1722, Final Elo: 1694.50, Average Points: 1.55, Winning Probability: 0.00
Player 2 - Initial Elo: 2261, Final Elo: 2239.87, Average Points: 3.60, Winning Probability: 0.35
Player 3 - Initial Elo: 1563, Final Elo: 1534.74, Average Points: 1.05, Winning Probability: 0.00
Player 4 - Initial Elo: 2314, Final Elo: 2335.21, Average Points: 3.69, Winning Probability: 0.30
Player 5 - Initial Elo: 2365, Final Elo: 2347.04, Average Points: 3.66, Winning Probability: 0.26
Player 6 - Initial Elo: 1752, Final Elo: 1761.65, Average Points: 1.94, Winning Probability: 0.00
Player 7 - Initial Elo: 1471, Final Elo: 1518.49, Average Points: 0.87, Winning Probability: 0.00
Player 8 - Initial Elo: 2050, Final Elo: 2044.62, Average Points: 2.81, Winning Probability: 0.04
Player 9 - Initial Elo: 2124, Final Elo: 2140.33, Average Points: 2.9

# Final code- 04 Dec

In [26]:
import random

# Number of Players, Rounds, and Simulations
num_players = 10
num_rounds = 5
num_simulations = 1000
k_factor = 10  # Higher K-factor for more responsive Elo rating updates

# Initialize Players with Elo ratings
players = [{'name': f'Player {i}', 'elo': random.randint(1200, 2000), 'points': 0, 'wins': 0, 'current_form': 0} for i in range(num_players)]

for player in players:
    print(f"{player ['name']} - Init Elo: {player['elo']:.2f}")
    
def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2, considering current form """
    # Adjusting Elo ratings based on current form
    adjusted_elo1 = player1['elo'] + player1['current_form']
    adjusted_elo2 = player2['elo'] + player2['current_form']
    return 1 / (1 + 10 ** ((adjusted_elo2 - adjusted_elo1) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings and current form after a game """
    expected_win_winner = calculate_win_probability(winner, loser)
    expected_win_loser = calculate_win_probability(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += k_factor * (score_winner - expected_win_winner)
    loser['elo'] += k_factor * (score_loser - expected_win_loser)

    # Update current form based on recent game outcome
    winner['current_form'] += k_factor * 1.2 if not draw else -k_factor * 0.3
    loser['current_form'] += -k_factor * 1.2 if not draw else k_factor * 0.5

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()
    if result < win_prob:
        player1['points'] += 1  # Player 1 wins
        update_elo_ratings(player1, player2)
    elif result < win_prob + (1 - win_prob) / 2:
#         print(player1['name'], player2['name'], " draw")
        player1['points'] += 0.5
        player2['points'] += 0.5  # Draw
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['points'] += 1  # Player 2 wins
        update_elo_ratings(player2, player1)

def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

def print_final_round_statistics(players, simulation_number):
    """ Print statistics for the final round """
    if 9990 <= simulation_number <= 10000:
        print(f"\nSimulation {simulation_number} - Final Round Results:")
        for player in players:
            winning_probability = player['wins'] / num_simulations
            print(f"{player['name']} - Points: {player['points']}, Elo: {player['elo']:.2f}, Winning Probability: {winning_probability:.2f}")

def simulate_tournament():
    """ Simulate a single Swiss-system tournament """
    # Reset points and opponents for each player
    for player in players:
        player['points'] = 0
        player['opponents'] = []
        player['current_form'] = 0

    # Simulate rounds
    for round_number in range(1, num_rounds + 1):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2)

    # Determine winner and update wins
    winner = max(players, key=lambda x: x['points'])
    winner['wins'] += 1
    
    
# Reset wins count before all simulations
for player in players:
    player['wins'] = 0

# Simulate all tournaments
for _ in range(1, num_simulations + 1):
    simulate_tournament()

    
# Calculate and print final probabilities after all simulations
print("\nFinal Tournament Results after all simulations:")
for player in players:
    winning_probability = player['wins'] / num_simulations
    print(f"{player['name']} - - Points: {player['points']} - - Final Elo: {player['elo']:.2f} - - Winning Probability: {winning_probability:.2f}")


Player 0 - Init Elo: 1556.00
Player 1 - Init Elo: 1548.00
Player 2 - Init Elo: 1991.00
Player 3 - Init Elo: 1214.00
Player 4 - Init Elo: 1232.00
Player 5 - Init Elo: 1698.00
Player 6 - Init Elo: 1396.00
Player 7 - Init Elo: 1531.00
Player 8 - Init Elo: 1751.00
Player 9 - Init Elo: 1707.00

Final Tournament Results after all simulations:
Player 0 - - Points: 4.5 - - Final Elo: 2165.36 - - Winning Probability: 0.76
Player 1 - - Points: 1 - - Final Elo: 1236.82 - - Winning Probability: 0.01
Player 2 - - Points: 2.5 - - Final Elo: 1606.58 - - Winning Probability: 0.12
Player 3 - - Points: 3 - - Final Elo: 1609.79 - - Winning Probability: 0.00
Player 4 - - Points: 4.5 - - Final Elo: 1989.47 - - Winning Probability: 0.05
Player 5 - - Points: 1 - - Final Elo: 1345.61 - - Winning Probability: 0.01
Player 6 - - Points: 2 - - Final Elo: 1662.80 - - Winning Probability: 0.01
Player 7 - - Points: 2.5 - - Final Elo: 1485.15 - - Winning Probability: 0.00
Player 8 - - Points: 3 - - Final Elo: 1722.38

In [2]:
import random

# Number of Players, Rounds, and Simulations
num_players = 10
num_rounds = 5
num_simulations = 1000
k_factor = 10  # Higher K-factor for more responsive Elo rating updates

# Initialize Players with Elo ratings
players = [{'name': f'Player {i}', 'elo': random.randint(1200, 2000), 'points': 0, 'wins': 0, 'current_form': 0, 'winning_probabilities': []} for i in range(num_players)]

def calculate_win_probability(player1, player2):
    """ Calculate the probability of player1 winning against player2, considering current form """
    adjusted_elo1 = player1['elo'] + player1['current_form']
    adjusted_elo2 = player2['elo'] + player2['current_form']
    return 1 / (1 + 10 ** ((adjusted_elo2 - adjusted_elo1) / 400))

def update_elo_ratings(winner, loser, draw=False):
    """ Update Elo ratings and current form after a game """
    expected_win_winner = calculate_win_probability(winner, loser)
    expected_win_loser = calculate_win_probability(loser, winner)

    score_winner = 0.5 if draw else 1
    score_loser = 0.5 if draw else 0

    winner['elo'] += k_factor * (score_winner - expected_win_winner)
    loser['elo'] += k_factor * (score_loser - expected_win_loser)

    winner['current_form'] += k_factor * 1.2 if not draw else -k_factor * 0.3
    loser['current_form'] += -k_factor * 1.2 if not draw else k_factor * 0.5

def simulate_game(player1, player2):
    """ Simulate a game between two players """
    win_prob = calculate_win_probability(player1, player2)
    result = random.random()

    if result < win_prob:
        player1['points'] += 1
        update_elo_ratings(player1, player2)
    elif result < win_prob + (1 - win_prob) / 2:
        player1['points'] += 0.5
        player2['points'] += 0.5
        update_elo_ratings(player1, player2, draw=True)
    else:
        player2['points'] += 1
        update_elo_ratings(player2, player1)

def swiss_pairing(players):
    """ Pair players using a simple Swiss system approach """
    sorted_players = sorted(players, key=lambda x: x['points'], reverse=True)
    pairings = []
    while sorted_players:
        player1 = sorted_players.pop(0)
        for i, player2 in enumerate(sorted_players):
            if not 'opponents' in player1 or player2['name'] not in player1['opponents']:
                player1.setdefault('opponents', []).append(player2['name'])
                player2.setdefault('opponents', []).append(player1['name'])
                pairings.append((player1, player2))
                sorted_players.pop(i)
                break
    return pairings

def simulate_tournament():
    """ Simulate a single Swiss-system tournament """
    for player in players:
        player['points'] = 0
        player['opponents'] = []
        player['current_form'] = 0
        player['winning_probabilities'] = []  # Clear winning probabilities

    for round_number in range(1, num_rounds + 1):
        pairings = swiss_pairing(players)
        for player1, player2 in pairings:
            simulate_game(player1, player2)
        
        # Calculate and store winning probabilities for each player after the round
        for player in players:
            total_wins_so_far = player['wins']
            total_simulations_so_far = round_number * num_simulations
            winning_probability = total_wins_so_far / total_simulations_so_far
            player
